# ETL

# Libraries

In [1]:
import os
import ast
import gc

import numpy as np
import pandas as pd

# Configurations

In [2]:
pd.set_option('display.max_columns', None)

pd.set_option('display.max_colwidth', None)

pd.set_option('display.width', 1000)

# Extract

Los dos datasets se descargan de esta dirección:

https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5

Los datasets deben estar en una carpeta llamada `raw` ubicada en la raiz del proyecto.

In [3]:
ruta_raw = os.path.abspath('../raw/')

## `movies`

In [4]:
movies = pd.read_csv(
    os.path.join(
        ruta_raw, 'movies_dataset.csv'))

C:\Users\franc\AppData\Local\Temp\ipykernel_15740\2227159362.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(


In [5]:
movies.head().T

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",NaN,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]","[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.","A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.","Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.","Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."


## `credits`

In [6]:
credits = pd.read_csv(
    os.path.join(
        ruta_raw, 'credits.csv'))

In [7]:
credits.head()

cast  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYG

# Transform

Transformaciones a realizar:

- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- Los valores nulos del campo release date deben eliminarse.

- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

## `movies`

In [8]:
movies.head(1).T

,0
adult,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
budget,30000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
homepage,http://toystory.disney.com/toy-story
id,862
imdb_id,tt0114709
original_language,en
original_title,Toy Story
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."


### Unnesting

#### belongs_to_collection

In [9]:
belongs_to_collection = movies[['id', 'belongs_to_collection']].copy()

movies = movies.drop(columns=['belongs_to_collection'])

In [10]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     45466 non-null  object
 1   belongs_to_collection  4494 non-null   object
dtypes: object(2)
memory usage: 710.5+ KB


In [11]:
belongs_to_collection.head(2)

,id,belongs_to_collection
0,862,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
1,8844,NaN


In [12]:
belongs_to_collection.rename(
    columns={
        'id': 'movie_id',
        'belongs_to_collection': 'collection_name'
    },
    inplace=True)

In [13]:
belongs_to_collection.head(2)

,movie_id,collection_name
0,862,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
1,8844,NaN


In [14]:
belongs_to_collection['collection_name'] = belongs_to_collection['collection_name'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

new_columns = pd.json_normalize(belongs_to_collection['collection_name'])

belongs_to_collection = belongs_to_collection.join(new_columns)

belongs_to_collection.drop(columns=['collection_name'], inplace=True)

del new_columns
gc.collect()

belongs_to_collection.rename(
    columns={
        'id': 'collection_id',
        'name': 'collection_name'
    },
    inplace=True)

In [15]:
belongs_to_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         45466 non-null  object 
 1   collection_id    4491 non-null   float64
 2   collection_name  4491 non-null   object 
 3   poster_path      3948 non-null   object 
 4   backdrop_path    3263 non-null   object 
dtypes: float64(1), object(4)
memory usage: 1.7+ MB


In [16]:
belongs_to_collection.head()

,movie_id,collection_id,collection_name,poster_path,backdrop_path
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,8844,NaN,NaN,NaN,NaN
2,15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,31357,NaN,NaN,NaN,NaN
4,11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [17]:
belongs_to_collection['collection_id'].value_counts()

collection_id
415931.0    29
421566.0    27
645.0       26
96887.0     26
37261.0     25
            ..
463112.0     1
474012.0     1
284700.0     1
142686.0     1
267551.0     1
Name: count, Length: 1695, dtype: int64

#### genres

In [18]:
genres = movies[['id', 'genres']].copy()

movies = movies.drop(columns=['genres'])

In [19]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45466 non-null  object
 1   genres  45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [20]:
genres.head(2)

,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"


In [21]:
genres.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [22]:
genres.head(2)

,movie_id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"


In [23]:
genres['genres'] = genres['genres'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = genres.explode('genres')

new_columns = pd.json_normalize(exploded['genres'])

genres = exploded.drop(columns=['genres']).reset_index(drop=True).join(new_columns)

In [24]:
del exploded
del new_columns
gc.collect()

286

In [25]:
genres.head(2)

,movie_id,id,name
0,862,16.0,Animation
1,862,35.0,Comedy


In [26]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93548 entries, 0 to 93547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  93548 non-null  object 
 1   id        91106 non-null  float64
 2   name      91106 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [27]:
genres.rename(
    columns={
        'id': 'genre_id',
        'name': 'genre_name'
    },
    inplace=True)

In [28]:
genres.columns

Index(['movie_id', 'genre_id', 'genre_name'], dtype='object')

In [29]:
genres['genre_id'].value_counts()

genre_id
18.0       20265
35.0       13182
53.0        7624
10749.0     6735
28.0        6596
27.0        4673
80.0        4307
99.0        3932
12.0        3496
878.0       3049
10751.0     2770
9648.0      2467
14.0        2313
16.0        1935
10769.0     1622
10402.0     1598
36.0        1398
10752.0     1323
37.0        1042
10770.0      767
11176.0        1
11602.0        1
29812.0        1
2883.0         1
7759.0         1
7760.0         1
7761.0         1
33751.0        1
17161.0        1
18012.0        1
18013.0        1
23822.0        1
Name: count, dtype: int64

#### production_companies

In [30]:
production_companies = movies[['id', 'production_companies']].copy()

movies = movies.drop(columns=['production_companies'])

In [31]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_companies  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [32]:
production_companies.head(2)

,id,production_companies
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"


In [33]:
production_companies.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [34]:
production_companies.columns

Index(['movie_id', 'production_companies'], dtype='object')

In [35]:
production_companies['production_companies'] = production_companies['production_companies'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_companies.explode('production_companies')

new_columns = pd.json_normalize(exploded['production_companies'])

production_companies = exploded.drop(columns=['production_companies']).reset_index(drop=True).join(new_columns)

In [36]:
del exploded
del new_columns
gc.collect()

275

In [37]:
production_companies.head(2)

,movie_id,name,id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0


In [38]:
production_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82426 entries, 0 to 82425
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  82426 non-null  object 
 1   name      70545 non-null  object 
 2   id        70545 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.9+ MB


In [39]:
production_companies.rename(
    columns={
        'id': 'company_id', 
        'name': 'company_name'
    },
    inplace=True)

In [40]:
production_companies.columns

Index(['movie_id', 'company_name', 'company_id'], dtype='object')

In [41]:
production_companies['company_name'].value_counts()

company_name
Warner Bros.                              1250
Metro-Goldwyn-Mayer (MGM)                 1076
Paramount Pictures                        1003
Twentieth Century Fox Film Corporation     836
Universal Pictures                         830
                                          ... 
Inside Track 1                               1
Film Council Premiere Fund                   1
Untitled 03                                  1
absolutely productions                       1
Yermoliev                                    1
Name: count, Length: 23537, dtype: int64

#### production_countries

In [42]:
production_countries = movies[['id', 'production_countries']].copy()

movies = movies.drop(columns=['production_countries'])

In [43]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    45466 non-null  object
 1   production_countries  45463 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [44]:
production_countries.head(2)

,id,production_countries
0,862,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"


In [45]:
production_countries.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [46]:
production_countries.columns

Index(['movie_id', 'production_countries'], dtype='object')

In [47]:
production_countries['production_countries'] = production_countries['production_countries'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = production_countries.explode('production_countries')

new_columns = pd.json_normalize(exploded['production_countries'])

production_countries = exploded.drop(columns=['production_countries']).reset_index(drop=True).join(new_columns)

In [48]:
del exploded
del new_columns
gc.collect()

264

In [49]:
production_countries.head(2)

,movie_id,iso_3166_1,name
0,862,US,United States of America
1,8844,US,United States of America


In [50]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55711 entries, 0 to 55710
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    55711 non-null  object
 1   iso_3166_1  49423 non-null  object
 2   name        49423 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [51]:
production_countries.rename(
    columns={
        'name': 'country_name'
    },
    inplace=True)

In [52]:
production_countries.columns

Index(['movie_id', 'iso_3166_1', 'country_name'], dtype='object')

In [53]:
production_countries['country_name'].value_counts()

country_name
United States of America    21153
United Kingdom               4094
France                       3940
Germany                      2254
Italy                        2169
                            ...  
Antarctica                      1
Gibraltar                       1
Brunei Darussalam               1
Honduras                        1
Guinea                          1
Name: count, Length: 160, dtype: int64

#### spoken_languages

In [54]:
spoken_languages = movies[['id', 'spoken_languages']].copy()

movies = movies.drop(columns=['spoken_languages'])

In [55]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                45466 non-null  object
 1   spoken_languages  45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [56]:
spoken_languages.head(2)

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]"


In [57]:
spoken_languages.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [58]:
spoken_languages.columns

Index(['movie_id', 'spoken_languages'], dtype='object')

In [59]:
spoken_languages['spoken_languages'] = spoken_languages['spoken_languages'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = spoken_languages.explode('spoken_languages')

new_columns = pd.json_normalize(exploded['spoken_languages'])

spoken_languages = exploded.drop(columns=['spoken_languages']).reset_index(drop=True).join(new_columns)

In [60]:
del exploded
del new_columns
gc.collect()

264

In [61]:
spoken_languages.head(2)

,movie_id,iso_639_1,name
0,862,en,English
1,8844,en,English


In [62]:
spoken_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57135 entries, 0 to 57134
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   57135 non-null  object
 1   iso_639_1  53300 non-null  object
 2   name       53300 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [63]:
spoken_languages.rename(
    columns={
        'name': 'language_name'
    },
    inplace=True)

In [64]:
spoken_languages.columns

Index(['movie_id', 'iso_639_1', 'language_name'], dtype='object')

In [65]:
spoken_languages['language_name'].value_counts()

language_name
English            28745
Français            4196
Deutsch             2625
Español             2413
Italiano            2367
                   ...  
Kinyarwanda            3
беларуская мова        2
Fulfulde               2
ozbek                  2
Hausa                  1
Name: count, Length: 75, dtype: int64

In [66]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   homepage           7782 non-null   object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   overview           44512 non-null  object 
 8   popularity         45461 non-null  object 
 9   poster_path        45080 non-null  object 
 10  release_date       45379 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

### Fill in the revenue and budget fields with 0

In [67]:
movies['revenue'].value_counts()

revenue
0.0           38052
12000000.0       20
10000000.0       19
11000000.0       19
2000000.0        18
              ...  
320395.0          1
50656.0           1
1328612.0         1
1268793.0         1
103504.0          1
Name: count, Length: 6863, dtype: int64

In [68]:
movies['budget'].value_counts()

budget
0           36573
5000000       286
10000000      259
20000000      243
2000000       242
            ...  
12200000        1
65700           1
64200           1
707503          1
8620000         1
Name: count, Length: 1226, dtype: int64

In [69]:
display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(6)

np.int64(0)

In [70]:
movies[['revenue', 'budget']] = movies[['revenue', 'budget']].fillna(0)

display(movies['revenue'].isna().sum())
display(movies['budget'].isna().sum())

np.int64(0)

np.int64(0)

### Remove null values ​​from the release_date field

In [71]:
movies['release_date'].isna().sum()

np.int64(87)

In [72]:
movies = movies.dropna(subset=['release_date'])

In [73]:
movies['release_date'].isna().sum()

np.int64(0)

### Format YYYY-mm-dd and create the release_year column

In [74]:
movies['release_date'].head(2)

0    1995-10-30
1    1995-12-15
Name: release_date, dtype: object

In [75]:
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [76]:
movies['release_date'].isna().sum()

np.int64(3)

In [77]:
movies[movies['release_date'].isna()]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'JP', 'name': 'Japan'}]",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
movies = movies.dropna(subset=['release_date'])

movies['release_date'].isna().sum()

np.int64(0)

In [79]:
movies['release_year'] = pd.to_datetime(movies['release_date']).dt.year

movies['release_year'].head(2)

0    1995
1    1995
Name: release_year, dtype: int32

### Create return column

In [80]:
movies['revenue'] = pd.to_numeric(movies['revenue']).astype(int)
movies['budget'] = pd.to_numeric(movies['budget']).astype(int)

movies['return'] = np.where((movies['revenue'] == 0) | (movies['budget'] == 0), 0, movies['revenue'] // movies['budget']).astype(int)

display(movies[['revenue', 'budget', 'return']].head(5))
print()
display(movies['return'].value_counts())

,revenue,budget,return
0,373554033,30000000,12
1,262797249,65000000,4
2,0,0,0
3,81452156,16000000,5
4,76578911,0,0


return
0       41600
1        1021
2         810
3         497
4         341
        ...  
426         1
69          1
2216        1
50          1
155         1
Name: count, Length: 113, dtype: int64

### Delete unnecessary columns

In [81]:
movies.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'release_year', 'return'], dtype='object')

In [82]:
movies = movies.drop(columns=["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

In [83]:
movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'release_year', 'return'], dtype='object')

## `credits`

In [84]:
credits.head(1).T

0
cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

#### Unnesting

##### `cast`

In [85]:
cast = credits[['id', 'cast']].copy()

cast.head(1).T

,0
id,862
cast,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg'}, {'cast_id': 22, 'character': 'Sid (voice)', 'credit_id': '52fe4284c3a36847f8024fb1', 'gender': 2, 'id': 12901, 'name': 'Erik von Detten', 'order': 8, 'profile_path': '/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg'}, {'cast_id': 23, 'character': 'Mrs. Davis (voice)', 'credit_id': '52fe4284c3a36847f8024fb5', 'gender': 1, 'id': 12133, 'name': 'Laurie Metcalf', 'order': 9, 'profile_path': '/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg'}, {'cast_id': 24, 'character': 'Sergeant (voice)', 'credit_id': '52fe4284c3a36847f8024fb9', 'gender': 2, 'id': 8655, 'name': 'R. Lee Ermey', 'order': 10, 'profile_path': '/r8GBqFBjypLUP9VVqDqfZ7wYbSs.jpg'}, {'cast_id': 25, 'character': 'Hannah (voice)', 'credit_id': '52fe4284c3a36847f8024fbd', 'gender': 1, 'id': 12903, 'name': 'Sarah Freeman', 'order': 11, 'profile_path': None}, {'cast_id': 27, 'character': 'TV Announcer (voice)', 'credit_id': '52fe4284c3a36847f8024fc5', 'gender': 2, 'id': 37221, 'name': 'Penn Jillette', 'order': 12, 'profile_path': '/zmAaXUdx12NRsssgHbk1T31j2x9.jpg'}]"


In [86]:
credits = credits.drop(columns=['cast'])

In [87]:
gc.collect()

17

In [88]:
cast.rename(
    columns={
        'id': 'movie_id'
    },
    inplace=True)

In [89]:
cast.columns

Index(['movie_id', 'cast'], dtype='object')

In [90]:
cast['cast'] = cast['cast'].apply(lambda x: ast.literal_eval(str(x)) if isinstance(x, str) and pd.notna(x) else x)

exploded = cast.explode('cast')

new_columns = pd.json_normalize(exploded['cast'])

cast = exploded.drop(columns=['cast']).reset_index(drop=True).join(new_columns)

In [91]:
del exploded
del new_columns
gc.collect()

154

In [92]:
cast.head(1).T

,0
movie_id,862
cast_id,14.0
character,Woody (voice)
credit_id,52fe4284c3a36847f8024f95
gender,2.0
id,31.0
name,Tom Hanks
order,0.0
profile_path,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg


In [93]:
cast.rename(
    columns={
        'id': 'actor_id'
    },
    inplace=True)

In [96]:
cast.columns

Index(['movie_id', 'cast_id', 'character', 'credit_id', 'gender', 'actor_id', 'name', 'order', 'profile_path'], dtype='object')

##### `crew`

In [97]:
crew = credits[['id', 'crew']].copy()

del credits
gc.collect()

crew.head(1).T

0
id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            